# Análise de Variância (ANOVA): Diferenciação Morfológica entre Espécies de Íris

**Autor:** Jesse Fernandes
**Data:** 2025-09-29  
**Objetivo**: Implementar e interpretar uma análise ANOVA rigorosa com pós-testes  
**Nível**: PhD/Pesquisa Avançada  
**Dataset**: Íris de Fisher (1936)

## 1. Fundamentação Teórica

### 1.1 Análise de Variância (ANOVA)

A análise de variância (ANOVA) é uma técnica estatística para avaliar diferenças entre médias de grupos em dados experimentais, desenvolvida por Sir Ronald Fisher. O teste realiza a comparação de médias através da análise da variância.

A hipótese nula ($H_0$) testada na ANOVA é:
$H_0: \mu_1 = \mu_2 = ... = \mu_k$

Contra a hipótese alternativa ($H_1$):
$H_1:$ pelo menos uma média difere das demais

A ANOVA decompõe a variância total dos dados em:
- **Variância entre grupos** (SSB): variabilidade entre médias de grupos diferentes
- **Variância dentro dos grupos** (SSW): variabilidade dentro de cada grupo

A estatística de teste F é calculada como:

$$F = \frac{MSB}{MSW} = \frac{SSB/(k-1)}{SSW/(N-k)}$$

onde:
- $k$ é o número de grupos
- $N$ é o número total de observações
- $MSB$ é a média quadrática entre grupos
- $MSW$ é a média quadrática dentro dos grupos

### 1.2 Suposições da ANOVA

1. **Independência**: As observações devem ser independentes entre si
2. **Normalidade**: Os dados em cada grupo devem seguir distribuição normal
3. **Homocedasticidade**: A variância deve ser homogênea entre os grupos

### 1.3 Testes Post-hoc

Quando a ANOVA rejeita $H_0$, os testes post-hoc identificam quais grupos diferem entre si:

- **Tukey HSD**: Controla a taxa de erro familiar, comparando todos os pares de médias
- **Bonferroni**: Ajusta o nível de significância para múltiplas comparações
- **Scheffé**: Mais conservador, adequado para comparações complexas de médias

## 2. Configuração do Ambiente Científico

In [ ]:
# Configuração do ambiente científico
import os
import sys
from pathlib import Path

# Configuração do projeto
project_root = Path(os.getcwd()).parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Imports científicos essenciais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from sklearn import datasets
import warnings

# Configurações de reprodutibilidade
np.random.seed(42)
warnings.filterwarnings('ignore')

# Configurações de visualização profissional
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (14, 8),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 18,
    'axes.grid': True,
    'grid.alpha': 0.3
})

print("✅ Ambiente científico configurado")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"SciPy: {stats.__version__ if hasattr(stats, '__version__') else 'N/A'}")
print(f"Statsmodels: {sm.__version__}")

: 

## 3. Carregamento e Exploração do Dataset

In [ ]:
class DataManager:
    """Classe para gerenciamento profissional de dados e análise exploratória."""
    
    def __init__(self, dataset_name="iris"):
        """Inicializa o gerenciador de dados com um dataset específico."""
        self.dataset_name = dataset_name
        self.data = None
        self.feature_names = None
        self.target_names = None
        
    def load_data(self):
        """Carrega o dataset e prepara para análise."""
        # Carregar dados
        if self.dataset_name == "iris":
            iris = datasets.load_iris()
            self.feature_names = iris.feature_names
            self.target_names = iris.target_names
            
            # Criar DataFrame
            self.data = pd.DataFrame(data=iris.data, columns=self.feature_names)
            self.data['species'] = [self.target_names[i] for i in iris.target]
            
            # Renomear colunas para formato mais adequado à análise estatística
            self.data.columns = [
                'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'
            ]
            
            print(f"✅ Dataset {self.dataset_name} carregado com sucesso")
            print(f"Dimensões: {self.data.shape[0]} observações × {self.data.shape[1]} variáveis")
            print(f"Espécies: {', '.join(self.data['species'].unique())}")
            print(f"Variáveis: {', '.join(self.data.columns[:-1])}")
            
            return self.data
        else:
            raise ValueError(f"Dataset {self.dataset_name} não suportado")
    
    def summarize_data(self):
        """Gera estatísticas descritivas do dataset."""
        if self.data is None:
            raise ValueError("Dataset não foi carregado. Execute load_data() primeiro.")
        
        # Estatísticas descritivas gerais
        print("\nEstatísticas Descritivas Gerais:")
        display(self.data.describe().round(2))
        
        # Estatísticas por espécie
        print("\nEstatísticas por Espécie:")
        stats_by_species = self.data.groupby('species').describe().round(2)
        display(stats_by_species)
        
        return stats_by_species
    
    def check_assumptions(self, var_name, alpha=0.05):
        """Verifica suposições estatísticas para ANOVA."""
        if self.data is None:
            raise ValueError("Dataset não foi carregado. Execute load_data() primeiro.")
        
        species = self.data['species'].unique()
        data_by_species = [self.data[self.data['species'] == sp][var_name] for sp in species]
        
        print(f"\n--- Verificação de Suposições para ANOVA: {var_name} ---\n")
        
        # 1. Teste de Normalidade (Shapiro-Wilk)
        print("1. Teste de Normalidade (Shapiro-Wilk):")
        all_normal = True
        for i, sp in enumerate(species):
            stat, p = stats.shapiro(data_by_species[i])
            normal = p > alpha
            all_normal &= normal
            print(f"   • {sp}: W = {stat:.4f}, p = {p:.4f} {'✓' if normal else '✗'}")
        
        # 2. Teste de Homogeneidade de Variâncias (Levene)
        stat, p = stats.levene(*data_by_species)
        homoscedasticity = p > alpha
        print(f"\n2. Teste de Homogeneidade de Variâncias (Levene):")
        print(f"   • W = {stat:.4f}, p = {p:.4f} {'✓' if homoscedasticity else '✗'}")
        
        # Conclusão sobre suposições
        print("\nConclusão sobre suposições:")
        
        if all_normal and homoscedasticity:
            print("✅ Todas as suposições para ANOVA paramétrica foram atendidas.")
            recommendation = "ANOVA paramétrica"
        elif homoscedasticity:
            print("⚠️ Suposição de normalidade violada, mas homogeneidade de variâncias atendida.")
            recommendation = "ANOVA robusta ou teste não-paramétrico (Kruskal-Wallis)"
        elif all_normal:
            print("⚠️ Suposição de homogeneidade de variâncias violada, mas normalidade atendida.")
            recommendation = "ANOVA com correção de Welch"
        else:
            print("❌ Suposições de normalidade e homogeneidade de variâncias violadas.")
            recommendation = "Teste não-paramétrico (Kruskal-Wallis)"
            
        print(f"Recomendação: {recommendation}")
        
        return {"normality": all_normal, "homoscedasticity": homoscedasticity, "recommendation": recommendation}
    
    def visualize_data(self):
        """Gera visualizações detalhadas para exploração dos dados."""
        if self.data is None:
            raise ValueError("Dataset não foi carregado. Execute load_data() primeiro.")
        
        # 1. Boxplots para cada variável por espécie
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, feature in enumerate(self.data.columns[:-1]):
            sns.boxplot(x='species', y=feature, data=self.data, ax=axes[i], palette='viridis')
            axes[i].set_title(f'Distribuição de {feature} por Espécie', fontsize=16)
            axes[i].set_xlabel('Espécie', fontsize=14)
            axes[i].set_ylabel(feature.replace('_', ' ').title(), fontsize=14)
        
        plt.tight_layout()
        plt.suptitle('Boxplots de Características por Espécie', fontsize=20, y=1.02)
        plt.show()
        
        # 2. Matriz de dispersão colorida por espécie
        plt.figure(figsize=(14, 10))
        sns.pairplot(self.data, hue='species', height=2.5, aspect=1.2, 
                     plot_kws={'alpha': 0.7, 's': 80, 'edgecolor': 'k', 'linewidth': 0.5},
                     diag_kind='kde', palette='viridis')
        plt.suptitle('Matriz de Dispersão de Características', fontsize=20, y=1.02)
        plt.tight_layout()
        plt.show()
        
        # 3. Histogramas de distribuição para cada variável por espécie
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, feature in enumerate(self.data.columns[:-1]):
            for species, color in zip(sorted(self.data['species'].unique()), 
                                      ['#440154', '#21918c', '#fde725']):
                sns.kdeplot(self.data[self.data['species'] == species][feature], 
                           ax=axes[i], label=species, fill=True, alpha=0.3)
                
            axes[i].set_title(f'Distribuição de {feature.replace("_", " ").title()}', fontsize=16)
            axes[i].set_xlabel(feature.replace('_', ' ').title(), fontsize=14)
            axes[i].set_ylabel('Densidade', fontsize=14)
            axes[i].legend()
        
        plt.tight_layout()
        plt.suptitle('Distribuição das Características por Espécie', fontsize=20, y=1.02)
        plt.show()

# Instanciar e utilizar o gerenciador de dados
data_mgr = DataManager(dataset_name="iris")
iris_df = data_mgr.load_data()
data_mgr.summarize_data()

In [ ]:
# Visualizar os dados
data_mgr.visualize_data()

## 4. Verificação de Suposições Estatísticas

In [ ]:
# Verificar suposições para cada variável
feature_assumptions = {}
for feature in iris_df.columns[:-1]:
    feature_assumptions[feature] = data_mgr.check_assumptions(feature)

## 5. Análise de Variância (ANOVA)

In [ ]:
class ANOVAAnalyzer:
    """Classe para execução rigorosa de análises ANOVA."""
    
    def __init__(self, data):
        """Inicializa o analisador ANOVA com um dataset específico."""
        self.data = data
        self.results = {}
        
    def run_oneway_anova(self, var_name, group_col='species', alpha=0.05):
        """Executa ANOVA unidirecional para uma variável específica."""
        print(f"\n--- ANOVA: {var_name} por {group_col} ---\n")
        
        # 1. ANOVA tradicional usando scipy.stats
        groups = [self.data[self.data[group_col] == g][var_name].values 
                 for g in self.data[group_col].unique()]
        
        f_stat, p_value = stats.f_oneway(*groups)
        significant = p_value < alpha
        
        print(f"Resultado da ANOVA Unidirecional:")
        print(f"F({len(groups)-1}, {self.data.shape[0]-len(groups)}) = {f_stat:.4f}, p = {p_value:.6f}")
        print(f"Conclusão: {'Rejeitar H₀' if significant else 'Não rejeitar H₀'} (α = {alpha})")
        
        if significant:
            print("Há evidências estatísticas de diferenças entre pelo menos duas espécies.")
        else:
            print("Não há evidências suficientes para afirmar diferenças entre as espécies.")
            
        # 2. ANOVA usando statsmodels (permite tabela ANOVA detalhada)
        formula = f"{var_name} ~ C({group_col})"
        model = ols(formula, data=self.data).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        
        print("\nTabela ANOVA Detalhada:")
        display(anova_table)
        
        # 3. Cálculo do tamanho do efeito (eta-squared)
        ss_total = anova_table['sum_sq'].sum()
        ss_between = anova_table['sum_sq'][0]
        eta_squared = ss_between / ss_total
        
        # Interpretar tamanho do efeito
        if eta_squared < 0.01:
            effect_interpretation = "negligenciável"
        elif eta_squared < 0.06:
            effect_interpretation = "pequeno"
        elif eta_squared < 0.14:
            effect_interpretation = "médio"
        else:
            effect_interpretation = "grande"
            
        print(f"\nTamanho do Efeito:")
        print(f"η² = {eta_squared:.4f} (efeito {effect_interpretation})")
        
        # 4. Power analysis (potência do teste)
        from statsmodels.stats.power import FTestAnovaPower
        power_analysis = FTestAnovaPower()
        groups_count = len(self.data[group_col].unique())
        n_per_group = self.data.groupby(group_col).size().mean()
        
        # Calcular f_effect (Cohen's f) a partir de eta_squared
        f_effect = np.sqrt(eta_squared / (1 - eta_squared))
        
        power = power_analysis.solve_power(
            effect_size=f_effect,
            nobs=self.data.shape[0],
            k_groups=groups_count,
            alpha=alpha
        )
        
        print(f"Potência do Teste: {power:.4f}")
        if power < 0.8:
            print(f"⚠️ Potência abaixo do recomendado (0.8). Há risco de erro Tipo II.")
            
            # Calcular tamanho da amostra necessário para potência 0.8
            required_n = power_analysis.solve_power(
                effect_size=f_effect,
                power=0.8,
                k_groups=groups_count,
                alpha=alpha
            )
            
            print(f"Para potência de 0.8, seriam necessárias aproximadamente {int(required_n)} observações totais.")
        else:
            print(f"✓ Potência adequada (>0.8). Baixo risco de erro Tipo II.")
        
        # Armazenar resultados
        self.results[var_name] = {
            'f_statistic': f_stat,
            'p_value': p_value,
            'significant': significant,
            'eta_squared': eta_squared,
            'effect_size_interpretation': effect_interpretation,
            'power': power,
            'model': model,
            'anova_table': anova_table
        }
        
        return self.results[var_name]
    
    def run_welch_anova(self, var_name, group_col='species', alpha=0.05):
        """Executa ANOVA de Welch (robusta a heterocedasticidade)."""
        print(f"\n--- ANOVA de Welch: {var_name} por {group_col} ---\n")
        
        # Preparar dados
        groups = self.data[group_col].unique()
        data_by_group = [self.data[self.data[group_col] == g][var_name] for g in groups]
        
        # Executar teste de Welch
        welch_results = stats.welch_anova(data_by_group)
        
        significant = welch_results[1] < alpha
        
        print(f"Resultado da ANOVA de Welch:")
        print(f"F({len(groups)-1}, {welch_results[2]:.2f}) = {welch_results[0]:.4f}, p = {welch_results[1]:.6f}")
        print(f"Conclusão: {'Rejeitar H₀' if significant else 'Não rejeitar H₀'} (α = {alpha})")
        
        # Armazenar resultados
        self.results[f"{var_name}_welch"] = {
            'f_statistic': welch_results[0],
            'p_value': welch_results[1],
            'significant': significant,
            'df_num': len(groups) - 1,
            'df_denom': welch_results[2]
        }
        
        return self.results[f"{var_name}_welch"]
    
    def run_kruskal_wallis(self, var_name, group_col='species', alpha=0.05):
        """Executa teste não-paramétrico de Kruskal-Wallis."""
        print(f"\n--- Teste de Kruskal-Wallis: {var_name} por {group_col} ---\n")
        
        # Preparar dados
        groups = self.data[group_col].unique()
        data_by_group = [self.data[self.data[group_col] == g][var_name] for g in groups]
        
        # Executar teste
        H_stat, p_value = stats.kruskal(*data_by_group)
        significant = p_value < alpha
        
        print(f"Resultado do Teste de Kruskal-Wallis:")
        print(f"H = {H_stat:.4f}, p = {p_value:.6f}")
        print(f"Conclusão: {'Rejeitar H₀' if significant else 'Não rejeitar H₀'} (α = {alpha})")
        
        # Calcular epsilon-squared como medida de tamanho de efeito
        n = len(self.data)
        epsilon_squared = (H_stat - len(groups) + 1) / (n - len(groups))
        
        # Interpretar tamanho de efeito
        if epsilon_squared < 0.01:
            effect_interpretation = "negligenciável"
        elif epsilon_squared < 0.06:
            effect_interpretation = "pequeno"
        elif epsilon_squared < 0.14:
            effect_interpretation = "médio"
        else:
            effect_interpretation = "grande"
            
        print(f"\nTamanho do Efeito:")
        print(f"ε² = {epsilon_squared:.4f} (efeito {effect_interpretation})")
        
        # Armazenar resultados
        self.results[f"{var_name}_kruskal"] = {
            'h_statistic': H_stat,
            'p_value': p_value,
            'significant': significant,
            'epsilon_squared': epsilon_squared,
            'effect_size_interpretation': effect_interpretation
        }
        
        return self.results[f"{var_name}_kruskal"]
    
    def post_hoc_tukey(self, var_name, group_col='species', alpha=0.05):
        """Executa teste post-hoc de Tukey HSD."""
        if var_name not in self.results:
            raise ValueError(f"ANOVA para {var_name} não foi executada. Execute run_oneway_anova primeiro.")
        
        if not self.results[var_name]['significant']:
            print(f"\nNota: ANOVA para {var_name} não foi significativa (p = {self.results[var_name]['p_value']:.4f}).")
            print("Testes post-hoc não são necessários, mas serão apresentados para fins didáticos.")
        
        print(f"\n--- Teste Post-hoc de Tukey HSD: {var_name} ---\n")
        
        # Executar teste de Tukey
        mc = MultiComparison(self.data[var_name], self.data[group_col])
        tukey_result = mc.tukeyhsd(alpha=alpha)
        
        print("Resultado do Teste de Tukey HSD:")
        print(tukey_result)
        
        # Matriz de p-valores em formato tabular
        try:
            print("\nMatriz de p-valores ajustados:")
            print(mc.groupsunique)
            print(tukey_result.pvalues)
        except:
            print("\nNota: Matriz de p-valores não disponível.")
        
        # Plotar resultado
        fig = plt.figure(figsize=(10, 6))
        tukey_result.plot_simultaneous(ax=plt.gca(), figsize=(10, 6))
        plt.title(f'Intervalos de Confiança (95%) para Diferenças em {var_name}', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Armazenar resultados
        self.results[f"{var_name}_tukey"] = {
            'tukey_result': tukey_result
        }
        
        return tukey_result

# Instanciar e utilizar o analisador ANOVA
anova_analyzer = ANOVAAnalyzer(iris_df)

# Executar ANOVA para comprimento da pétala
anova_analyzer.run_oneway_anova('petal_length')
anova_analyzer.post_hoc_tukey('petal_length')

In [ ]:
# Executar ANOVA para largura da pétala
anova_analyzer.run_oneway_anova('petal_width')
anova_analyzer.post_hoc_tukey('petal_width')

In [ ]:
# Executar ANOVA para comprimento da sépala
anova_analyzer.run_oneway_anova('sepal_length')
anova_analyzer.post_hoc_tukey('sepal_length')

In [ ]:
# Executar ANOVA para largura da sépala
anova_analyzer.run_oneway_anova('sepal_width')
anova_analyzer.post_hoc_tukey('sepal_width')

In [ ]:
# Executar análises alternativas para variáveis que não atendem às suposições
for feature, assumption in feature_assumptions.items():
    if not (assumption['normality'] and assumption['homoscedasticity']):
        print(f"\n--- Análise Alternativa para {feature} ---")
        print(f"Recomendação: {assumption['recommendation']}")
        
        if assumption['recommendation'] == 'ANOVA com correção de Welch':
            anova_analyzer.run_welch_anova(feature)
        elif 'não-paramétrico' in assumption['recommendation']:
            anova_analyzer.run_kruskal_wallis(feature)

## 6. Visualização dos Resultados

In [ ]:
class ResultVisualizer:
    """Classe para visualização profissional dos resultados de ANOVA."""
    
    def __init__(self, data, anova_results):
        """Inicializa o visualizador com dados e resultados de ANOVA."""
        self.data = data
        self.results = anova_results
    
    def plot_means_with_ci(self, var_name, group_col='species'):
        """Plota médias com intervalos de confiança por grupo."""
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Estatísticas por grupo
        groups = []
        means = []
        ci_low = []
        ci_high = []
        
        for group in sorted(self.data[group_col].unique()):
            group_data = self.data[self.data[group_col] == group][var_name]
            groups.append(group)
            
            # Média
            mean = group_data.mean()
            means.append(mean)
            
            # Intervalo de confiança de 95%
            ci = stats.t.interval(
                0.95, 
                len(group_data) - 1, 
                loc=mean, 
                scale=stats.sem(group_data)
            )
            ci_low.append(ci[0])
            ci_high.append(ci[1])
        
        # Plotar barras
        x = np.arange(len(groups))
        width = 0.6
        
        # Barras de erro com médias
        bars = ax.bar(x, means, width, capsize=10, 
                     yerr=np.vstack((np.array(means) - np.array(ci_low), 
                                    np.array(ci_high) - np.array(means))),
                     color=['#440154', '#21918c', '#fde725'], 
                     alpha=0.8, edgecolor='black', linewidth=1)
        
        # Adicionar valor da média
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                   f'{means[i]:.2f}', ha='center', va='bottom', fontsize=12)
        
        # Configurar eixos
        ax.set_ylabel(var_name.replace('_', ' ').title(), fontsize=14)
        ax.set_xticks(x)
        ax.set_xticklabels(groups, fontsize=12)
        
        # Adicionar título com resultados da ANOVA
        if var_name in self.results:
            result = self.results[var_name]
            title = f"Médias de {var_name.replace('_', ' ').title()} por Espécie\n"
            title += f"ANOVA: F({len(groups)-1}, {self.data.shape[0]-len(groups)}) = {result['f_statistic']:.2f}, "
            title += f"p = {result['p_value']:.6f}, η² = {result['eta_squared']:.3f}"
            ax.set_title(title, fontsize=16)
        else:
            ax.set_title(f"Médias de {var_name.replace('_', ' ').title()} por Espécie", fontsize=16)
        
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    
    def plot_all_means(self):
        """Plota médias de todas as variáveis em um grid."""
        features = [col for col in self.data.columns if col != 'species']
        
        fig, axes = plt.subplots(2, 2, figsize=(18, 14))
        axes = axes.flatten()
        
        for i, feature in enumerate(features):
            ax = axes[i]
            
            # Dados para o gráfico
            stats_by_group = self.data.groupby('species')[feature].agg(['mean', 'std', 'count'])
            groups = stats_by_group.index
            means = stats_by_group['mean'].values
            
            # Calcular erro padrão
            std_err = stats_by_group['std'] / np.sqrt(stats_by_group['count'])
            
            # Calcular intervalo de confiança de 95%
            ci_low = []
            ci_high = []
            
            for j, group in enumerate(groups):
                t_val = stats.t.ppf(0.975, stats_by_group['count'][j] - 1)
                ci_low.append(means[j] - t_val * std_err[j])
                ci_high.append(means[j] + t_val * std_err[j])
            
            # Plotar barras
            x = np.arange(len(groups))
            width = 0.6
            
            # Barras de erro com médias
            bars = ax.bar(x, means, width, capsize=10, 
                         yerr=np.vstack((np.array(means) - np.array(ci_low), 
                                        np.array(ci_high) - np.array(means))),
                         color=['#440154', '#21918c', '#fde725'], 
                         alpha=0.8, edgecolor='black', linewidth=1)
            
            # Adicionar valor da média
            for j, bar in enumerate(bars):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.02*(ax.get_ylim()[1]-ax.get_ylim()[0]),
                       f'{means[j]:.2f}', ha='center', va='bottom', fontsize=11)
            
            # Configurar eixos
            ax.set_ylabel(feature.replace('_', ' ').title(), fontsize=14)
            ax.set_xticks(x)
            ax.set_xticklabels(groups, fontsize=12)
            
            # Adicionar título com resultados da ANOVA
            if feature in self.results:
                result = self.results[feature]
                title = f"{feature.replace('_', ' ').title()}\n"
                title += f"F = {result['f_statistic']:.1f}, p = {result['p_value']:.4f}"
                ax.set_title(title, fontsize=14)
            else:
                ax.set_title(feature.replace('_', ' ').title(), fontsize=14)
            
            ax.grid(axis='y', linestyle='--', alpha=0.7)
        
        plt.suptitle('Comparação de Médias entre Espécies', fontsize=20, y=0.98)
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        plt.show()
    
    def summarize_results(self):
        """Sumariza os resultados de todas as análises em formato tabular."""
        # Coletar resultados
        summary_data = []
        
        for var_name, result in self.results.items():
            if var_name.endswith('_welch') or var_name.endswith('_kruskal') or var_name.endswith('_tukey'):
                continue
                
            row = {
                'Variável': var_name.replace('_', ' ').title(),
                'F-valor': result['f_statistic'],
                'p-valor': result['p_value'],
                'Significância': 'Significativo' if result['significant'] else 'Não significativo',
                'η²': result.get('eta_squared', None),
                'Tamanho do Efeito': result.get('effect_size_interpretation', 'N/A'),
                'Potência': result.get('power', None)
            }
            summary_data.append(row)
        
        # Criar DataFrame resumo
        summary_df = pd.DataFrame(summary_data)
        summary_df = summary_df.sort_values('p-valor')
        
        print("\n=== Sumário dos Resultados da ANOVA ===\n")
        display(summary_df.reset_index(drop=True))
        
        # Interpretação geral
        print("\nInterpretação Geral:")
        sig_vars = summary_df[summary_df['Significância'] == 'Significativo']['Variável'].tolist()
        
        if len(sig_vars) > 0:
            print(f"• As seguintes variáveis apresentam diferenças significativas entre espécies: {', '.join(sig_vars)}.")
            
            # Identificar maior efeito
            if 'η²' in summary_df.columns:
                max_effect_var = summary_df.loc[summary_df['η²'].idxmax()]['Variável']
                max_effect = summary_df.loc[summary_df['η²'].idxmax()]['η²']
                print(f"• A variável {max_effect_var} apresenta o maior efeito (η² = {max_effect:.4f}).")
        else:
            print("• Nenhuma variável apresentou diferenças significativas entre espécies.")
            
        # Plotar heatmap de significância
        self.plot_significance_heatmap()
            
        return summary_df
        
    def plot_significance_heatmap(self):
        """Cria um mapa de calor de significância para comparações post-hoc."""
        # Verificar quais variáveis têm resultados Tukey
        features_with_tukey = [var.replace('_tukey', '') 
                             for var in self.results.keys() 
                             if var.endswith('_tukey')]
        
        if not features_with_tukey:
            print("Não há resultados post-hoc disponíveis para visualização.")
            return
            
        # Preparar dados para o heatmap
        species = sorted(self.data['species'].unique())
        n_species = len(species)
        n_features = len(features_with_tukey)
        
        # Criar matrizes vazias
        pvalues = np.ones((n_features, n_species, n_species)) * np.nan
        significance = np.zeros((n_features, n_species, n_species), dtype=int)
        
        # Preencher com resultados
        for f_idx, feature in enumerate(features_with_tukey):
            if f"{feature}_tukey" in self.results:
                tukey_result = self.results[f"{feature}_tukey"]['tukey_result']
                
                for i, group1 in enumerate(tukey_result.groupsunique):
                    for j, group2 in enumerate(tukey_result.groupsunique):
                        if i < j:  # Preencher apenas o triângulo superior
                            # Encontrar índice de comparação
                            comp_idx = None
                            for k, (g1, g2, _) in enumerate(tukey_result._multicomp.pairindices):
                                if (g1 == i and g2 == j) or (g1 == j and g2 == i):
                                    comp_idx = k
                                    break
                            
                            if comp_idx is not None:
                                p_value = tukey_result.pvalues[comp_idx]
                                pvalues[f_idx, i, j] = p_value
                                pvalues[f_idx, j, i] = p_value  # Simetria
                                
                                # Codificar significância: 0=NS, 1=p<0.05, 2=p<0.01, 3=p<0.001
                                if p_value < 0.001:
                                    sig_code = 3
                                elif p_value < 0.01:
                                    sig_code = 2
                                elif p_value < 0.05:
                                    sig_code = 1
                                else:
                                    sig_code = 0
                                    
                                significance[f_idx, i, j] = sig_code
                                significance[f_idx, j, i] = sig_code  # Simetria
        
        # Plotar heatmaps
        fig, axes = plt.subplots(1, n_features, figsize=(5*n_features, 5))
        if n_features == 1:
            axes = [axes]
            
        # Mapa de cores para significância
        cmap = plt.cm.get_cmap('YlOrRd', 4)
        
        for f_idx, (feature, ax) in enumerate(zip(features_with_tukey, axes)):
            # Criar matriz para plotar
            sig_matrix = pd.DataFrame(significance[f_idx], 
                                     index=species, 
                                     columns=species)
            
            # Plot heatmap
            sns.heatmap(sig_matrix, annot=True, cmap=cmap, 
                       cbar_kws={'ticks': [0, 1, 2, 3]},
                       vmin=0, vmax=3, ax=ax, fmt='d')
            
            # Configurações
            ax.set_title(f"{feature.replace('_', ' ').title()}")
            
            # Ajustar diagonais
            for i in range(len(species)):
                sig_matrix.iloc[i, i] = np.nan
        
        # Adicionar legenda de cores
        plt.tight_layout()
        fig.subplots_adjust(bottom=0.2)
        
        # Legenda personalizada
        legend_labels = ["NS: p≥0.05", "*: p<0.05", "**: p<0.01", "***: p<0.001"]
        custom_lines = [plt.Line2D([0], [0], color=cmap(i/3), lw=7) for i in range(4)]
        fig.legend(custom_lines, legend_labels, loc='lower center', ncol=4, fontsize=12)
        
        plt.suptitle("Significância das Diferenças entre Espécies (Tukey HSD)", fontsize=18, y=1.05)
        plt.tight_layout()
        plt.show()

# Criar e utilizar o visualizador de resultados
visualizer = ResultVisualizer(iris_df, anova_analyzer.results)
visualizer.plot_all_means()
visualizer.summarize_results()

## 7. Análise de Componentes Principais (PCA)

Como análise complementar, podemos visualizar o agrupamento das espécies usando PCA para verificar como as diferenças identificadas pela ANOVA se manifestam em termos de variabilidade multivariada.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Preparar dados para PCA
X = iris_df.iloc[:, :-1].values  # Todas as features
y = iris_df['species'].values    # Classes

# Padronizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Variância explicada
explained_variance = pca.explained_variance_ratio_ * 100
cumulative_variance = np.cumsum(explained_variance)

# Plotar variância explicada
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.8, 
       color='skyblue', edgecolor='black', label='Variância individual')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', 
        color='red', label='Variância acumulada')
plt.axhline(y=95, color='k', linestyle='--', alpha=0.5, label='Limite 95%')
plt.xlabel('Componentes Principais', fontsize=14)
plt.ylabel('% de Variância Explicada', fontsize=14)
plt.title('Variância Explicada por Componente Principal', fontsize=16)
plt.xticks(range(1, len(explained_variance) + 1))
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plotar os resultados do PCA (primeiros 2 componentes)
plt.figure(figsize=(12, 10))

# Cores por espécie
colors = {'setosa': '#440154', 'versicolor': '#21918c', 'virginica': '#fde725'}

# Plotar pontos
for species in np.unique(y):
    plt.scatter(X_pca[y == species, 0], X_pca[y == species, 1], s=100, alpha=0.8,
               edgecolor='k', linewidth=1, color=colors[species], label=species)

# Adicionar vetores de features
features = iris_df.columns[:-1]
for i, feature in enumerate(features):
    plt.arrow(0, 0, pca.components_[0, i]*7, pca.components_[1, i]*7,
             head_width=0.2, head_length=0.2, fc='k', ec='k')
    plt.text(pca.components_[0, i]*7.5, pca.components_[1, i]*7.5, feature,
            fontsize=14)

# Adicionar círculo de correlações
circle = plt.Circle((0, 0), 1, fill=False, color='gray', linestyle='--')
plt.gca().add_patch(circle)

# Configurações do gráfico
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel(f'PC1 ({explained_variance[0]:.1f}%)', fontsize=14)
plt.ylabel(f'PC2 ({explained_variance[1]:.1f}%)', fontsize=14)
plt.title('Análise de Componentes Principais: Espécies de Íris', fontsize=18)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=14)

# Ajustar limites para incluir vetores e rótulos
plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.tight_layout()
plt.show()

# Exibir coeficientes (loadings) dos componentes principais
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(pca.components_.shape[0])],
    index=features
)
print("Coeficientes dos Componentes Principais (loadings):")
display(loadings)

## 8. Conclusões e Interpretação Biológica

### 8.1 Síntese dos Resultados

1. **Diferenças Morfológicas**:
   - As características da pétala (comprimento e largura) exibem as diferenças mais pronunciadas entre espécies (tamanho de efeito grande)
   - O comprimento da sépala também apresenta diferenças significativas entre as três espécies
   - A largura da sépala exibe o menor poder discriminatório, embora ainda apresente diferenças significativas

2. **Padrões de Variabilidade**:
   - *Iris setosa* mostra clara separação morfológica das outras duas espécies
   - *Iris versicolor* e *Iris virginica* apresentam alguma sobreposição, especialmente nas características da sépala
   - A análise PCA confirma essa separação, com os dois primeiros componentes explicando aproximadamente 95% da variância

3. **Importância Taxonômica**:
   - O comprimento da pétala emerge como o indicador mais confiável para classificação taxonômica destas espécies
   - Combinações de características, conforme mostrado pela PCA, oferecem maior poder discriminatório que características isoladas

### 8.2 Limitações do Estudo

1. O tamanho amostral, embora adequado para ANOVA (potência > 0.8), poderia ser ampliado para capturar maior variabilidade intraespecífica
2. As características analisadas limitam-se a medidas morfológicas; análises adicionais poderiam incluir características ecológicas ou genéticas
3. A presença de alguns outliers pode influenciar ligeiramente as estatísticas paramétricas

### 8.3 Recomendações e Estudos Futuros

1. Desenvolver modelos de classificação baseados nas diferenças morfométricas identificadas
2. Investigar correlações entre características morfológicas e fatores ecológicos (habitat, polinizadores)
3. Expandir a análise para incluir mais espécies do gênero Iris e explorar relações filogenéticas
4. Aplicar técnicas de morfometria geométrica para análise mais detalhada da forma das pétalas e sépalas

A análise ANOVA fornece evidências estatísticas robustas de que as três espécies de íris são morfologicamente distintas, corroborando a classificação taxonômica de Fisher e demonstrando a utilidade das técnicas de análise de variância em estudos de biodiversidade vegetal.